In [1]:
import pandas as pd
import numpy as np
import heapq
import math
import time

import gmpy2
from gmpy2 import mpz
import re

from sklearn import tree

import cProfile

In [2]:
# Read in the dataset
df = pd.DataFrame(pd.read_csv('../data/compas-binary.csv'))

In [3]:
x_all = df.as_matrix()[:,:13]

y = df.as_matrix()[:,13]

In [4]:
# Association Rule Mining (Only one feature)

#support
supp1 = [(x_all[:,i]*y).mean() for i in range(13)]
supp1

[0.07050818010713769,
 0.05993919212393224,
 0.049804546112639354,
 0.06906037353409585,
 0.2265817286810482,
 0.057333140292456924,
 0.4327493846822065,
 0.420587809468655,
 0.36557115969306503,
 0.09497611119154481,
 0.08817142029824815,
 0.07137686405096279,
 0.20819458520341683]

In [5]:
#support
supp0 = [((1-x_all[:,i])*y).mean() for i in range(13)]
supp0

[0.3922108006370349,
 0.40277978862024033,
 0.4129144346315332,
 0.3936586072100767,
 0.23613725206312436,
 0.40538584045171566,
 0.02996959606196612,
 0.04213117127551759,
 0.09714782105110757,
 0.36774286955262775,
 0.3745475604459244,
 0.3913421166932098,
 0.25452439554075573]

In [6]:
#confidence
conf1 = [sum(x_all[:,i]*y)/sum(x_all[:,i]) for i in range(13)]
conf1

[0.3667168674698795,
 0.592274678111588,
 0.5300462249614792,
 0.5184782608695652,
 0.45773618016964024,
 0.32459016393442625,
 0.45069360675512665,
 0.44644229291532195,
 0.4233735747820255,
 0.4932330827067669,
 0.28986197049024276,
 0.37864823348694315,
 0.6614535418583257]

In [7]:
#confidence
conf0 = [sum((x_all[:,i]==0)*y)/sum((x_all[:,i]==0)) for i in range(13)]
conf0

[0.48557089084065247,
 0.4481314432989691,
 0.45573665707893896,
 0.45415065976281943,
 0.4676032110091743,
 0.4923509759099701,
 0.7527272727272727,
 0.7275,
 0.711558854718982,
 0.45544199390353235,
 0.5382854764877236,
 0.4822479928635147,
 0.37143460807099093]

In [8]:
x_idx_mine = [(conf1[i]>=0.5) or (conf0[i]>=0.5) for i in range(13)]
x_idx_mine

[False,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 True,
 False,
 True,
 False,
 True]

In [9]:
x_idx_mine[9]=True
x_idx_mine
x_mine = x_all[:,x_idx_mine]
x_idx_mine

[False,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 False,
 True]

In [10]:
#manaually select out 5 features, accoring to CORELS paper when lambda=0.01
## sex:Female, age:18-20,age:21-22, juvenile-crimes:=0, priors:>3
##x_idx = [0,1,2,8,12]

# sex:Female, age:18-20,age:21-22, priors:2-3, priors:>3
x_idx = [0,1,2,9,12]
x = x_all[:,x_idx]
x

array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 1, 0, 1],
       ...,
       [0, 0, 0, 0, 0],
       [1, 0, 0, 1, 0],
       [1, 0, 1, 1, 0]])

In [11]:
#manaually select out 6 features, accoring to CORELS paper when lambda=0.01
# sex:Female, age:18-20,age:21-22, juvenile-crimes:=0, priors:2-3, priors:>3
x_idx6 = [0,1,2,8,9,12]
x6 = x_all[:,x_idx6]
x6

array([[0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 1],
       ...,
       [0, 0, 0, 1, 0, 0],
       [1, 0, 0, 1, 1, 0],
       [1, 0, 1, 1, 1, 0]])

In [12]:
nrule = x.shape[1]
ndata = len(y)

In [13]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(x, y)

In [14]:
accu = clf.score(x,y)
nleaves = (clf.tree_.node_count+1)/2
R_c = 1-accu + 0.001*nleaves
R_c

0.3520089764007529

In [15]:
print(clf.tree_.node_count) #get the node count

35


In [16]:
# manually add a highly correlated new feature, to test the similar support bound
import random
row_idx = random.sample(range(x.shape[0]),600)
new_feature = x[:,2].copy()
new_feature[row_idx] = 1-new_feature[row_idx]
x_corr = np.column_stack((x, new_feature))

In [17]:
pd.DataFrame(x_corr).corr()

,0,1,2,3,4,5
0,1.000000,-0.021191,-0.002244,-0.009986,-0.107579,-0.001743
1,-0.021191,1.000000,-0.108061,-0.017773,-0.194341,-0.073210
2,-0.002244,-0.108061,1.000000,0.039048,-0.106069,0.652407
3,-0.009986,-0.017773,0.039048,1.000000,-0.330969,0.046345
4,-0.107579,-0.194341,-0.106069,-0.330969,1.000000,-0.069429
5,-0.001743,-0.073210,0.652407,0.046345,-0.069429,1.000000


## With similar support bound

In [18]:
from corels_dt_nosimilar import bbound_nosimilar
from corels_dt_similar import bbound_similar
from corels_dt_similar_priority import bbound_similar_priority
from corels_dt_similar_when_big import bbound_similar_when_big

from corels_dt_similar_when_sub_v2 import bbound_similar_when_sub

from corels_dt_nosimilar_multicopies import bbound_nosimilar_multicopies
from corels_dt_nosimilar_multicopies0 import bbound_nosimilar_multicopies0

In [19]:
#10.15
bbound_nosimilar_multicopies(x, y, lamb=0.0035, prior_metric="bound", MAXDEPTH = 5)

nrule: 5
ndata: 6907
the rank of x's columns:  [4, 0, 1, 2, 3]
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  7.236100673675537
lambda:  0.0035
leaves:  [(3,), (-3, 1), (-5, -3, -1), (-4, -3, -1, 5), (-3, -1, 4, 5)]
prediction:  [1, 1, 0, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  22194
time when the best tree is achieved:  7.119937896728516
TOTAL COUNT:  22211


In [19]:
#10.15
bbound_nosimilar_multicopies(x, y, lamb=0.0035, prior_metric="objective", MAXDEPTH = 5)

nrule: 5
ndata: 6907
the rank of x's columns:  [4, 0, 1, 2, 3]
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  4.774858713150024
lambda:  0.0035
leaves:  [(1,), (-1, 3), (-5, -3, -1), (-4, -3, -1, 5), (-3, -1, 4, 5)]
prediction:  [1, 1, 0, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  516
time when the best tree is achieved:  2.2769076824188232
TOTAL COUNT:  1672


In [20]:
#10.15
bbound_nosimilar_multicopies(x, y, lamb=0.0035, prior_metric="curiosity", MAXDEPTH = 5)

nrule: 5
ndata: 6907
the rank of x's columns:  [4, 0, 1, 2, 3]
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  5.902628421783447
lambda:  0.0035
leaves:  [(1,), (-4, -3, -1), (-4, -1, 3), (-5, -1, 4), (-1, 4, 5)]
prediction:  [1, 0, 1, 0, 1]
Objective:  0.35411502823222823
COUNT of the best tree:  137
time when the best tree is achieved:  1.6367299556732178
TOTAL COUNT:  7172


In [21]:
#10.15
bbound_nosimilar_multicopies(x, y, lamb=0.0035, prior_metric="gini", MAXDEPTH = 5)

nrule: 5
ndata: 6907
the rank of x's columns:  [4, 0, 1, 2, 3]
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  6.1913902759552
lambda:  0.0035
leaves:  [(1,), (-1, 3), (-5, -3, -1), (-4, -3, -1, 5), (-3, -1, 4, 5)]
prediction:  [1, 1, 0, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  6439
time when the best tree is achieved:  5.374780893325806
TOTAL COUNT:  9138


In [22]:
#10.15
bbound_nosimilar_multicopies(x6, y, lamb=0.0035, prior_metric="objective", MAXDEPTH = 5)

nrule: 6
ndata: 6907
the rank of x's columns:  [5, 0, 4, 1, 2, 3]
COUNT: 100000
COUNT: 200000
COUNT: 300000
COUNT: 400000
COUNT: 500000
COUNT: 600000
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  257.9732565879822
lambda:  0.0035
leaves:  [(1,), (-1, 4), (-4, -3, -1), (-5, -4, -1, 3), (-4, -1, 3, 5)]
prediction:  [1, 1, 0, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  51646
time when the best tree is achieved:  10.576807498931885
TOTAL COUNT:  657173


In [21]:
bbound_nosimilar_multicopies0(x6, y, lamb=0.0035, prior_metric="objective", MAXDEPTH = 5)

nrule: 6
ndata: 6907
the rank of x's columns:  [5, 0, 4, 1, 2, 3]
COUNT: 100000
COUNT: 200000
COUNT: 300000
COUNT: 400000
COUNT: 500000
COUNT: 600000
COUNT: 700000
COUNT: 800000
COUNT: 900000
COUNT: 1000000
COUNT: 1100000
COUNT: 1200000
COUNT: 1300000
COUNT: 1400000
COUNT: 1500000
COUNT: 1600000
COUNT: 1700000
COUNT: 1800000
COUNT: 1900000
COUNT: 2000000
COUNT: 2100000
COUNT: 2200000
COUNT: 2300000
COUNT: 2400000
COUNT: 2500000
COUNT: 2600000
COUNT: 2700000
COUNT: 2800000
COUNT: 2900000
COUNT: 3000000
COUNT: 3100000
COUNT: 3200000
COUNT: 3300000
COUNT: 3400000
COUNT: 3500000
COUNT: 3600000
COUNT: 3700000
COUNT: 3800000
COUNT: 3900000
COUNT: 4000000
COUNT: 4100000
COUNT: 4200000
COUNT: 4300000
COUNT: 4400000
COUNT: 4500000
COUNT: 4600000
COUNT: 4700000
COUNT: 4800000
COUNT: 4900000
COUNT: 5000000
COUNT: 5100000
COUNT: 5200000
COUNT: 5300000
COUNT: 5400000
COUNT: 5500000
COUNT: 5600000
COUNT: 5700000
COUNT: 5800000
COUNT: 5900000
COUNT: 6000000
COUNT: 6100000
COUNT: 6200000
COUNT: 630000

In [23]:
#10.15
bbound_nosimilar_multicopies(x6, y, lamb=0.0035, prior_metric="curiosity", MAXDEPTH = 5)

nrule: 6
ndata: 6907
the rank of x's columns:  [5, 0, 4, 1, 2, 3]
COUNT: 100000
COUNT: 200000
COUNT: 300000
COUNT: 400000
COUNT: 500000
COUNT: 600000
COUNT: 700000
COUNT: 800000
COUNT: 900000
COUNT: 1000000
COUNT: 1100000
COUNT: 1200000
COUNT: 1300000
COUNT: 1400000
COUNT: 1500000
COUNT: 1600000
COUNT: 1700000
COUNT: 1800000
COUNT: 1900000
COUNT: 2000000
COUNT: 2100000
COUNT: 2200000
COUNT: 2300000
COUNT: 2400000
COUNT: 2500000
COUNT: 2600000
COUNT: 2700000
COUNT: 2800000
COUNT: 2900000
COUNT: 3000000
COUNT: 3100000
COUNT: 3200000
COUNT: 3300000
COUNT: 3400000
COUNT: 3500000
COUNT: 3600000
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  1272.7291719913483
lambda:  0.0035
leaves:  [(1,), (-1, 4), (-4, -3, -1), (-5, -4, -1, 3), (-4, -1, 3, 5)]
prediction:  [1, 1, 0, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  2585
time when the best tree is achieved:  7.540153980255127
TOTAL COUN

In [1]:
bbound_nosimilar_multicopies0(x6, y, lamb=0.0035, prior_metric="curiosity", MAXDEPTH = 5)

NameError: name 'bbound_nosimilar_multicopies0' is not defined

In [24]:
#10.15
bbound_nosimilar_multicopies(x6, y, lamb=0.0035, prior_metric="gini", MAXDEPTH = 5)

nrule: 6
ndata: 6907
the rank of x's columns:  [5, 0, 4, 1, 2, 3]
COUNT: 100000
COUNT: 200000
COUNT: 300000
COUNT: 400000
COUNT: 500000
COUNT: 600000
COUNT: 700000
COUNT: 800000
COUNT: 900000
COUNT: 1000000
COUNT: 1100000
COUNT: 1200000
COUNT: 1300000
COUNT: 1400000
COUNT: 1500000
COUNT: 1600000
COUNT: 1700000
COUNT: 1800000
COUNT: 1900000
COUNT: 2000000
COUNT: 2100000
COUNT: 2200000
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  900.7868373394012
lambda:  0.0035
leaves:  [(1,), (-1, 4), (-4, -3, -1), (-5, -4, -1, 3), (-4, -1, 3, 5)]
prediction:  [1, 1, 0, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  1452590
time when the best tree is achieved:  481.41348099708557
TOTAL COUNT:  2253718


In [ ]:
bbound_nosimilar_multicopies0(x6, y, lamb=0.0035, prior_metric="gini", MAXDEPTH = 5)

nrule: 6
ndata: 6907
the rank of x's columns:  [5, 0, 4, 1, 2, 3]
COUNT: 100000
COUNT: 200000
COUNT: 300000
COUNT: 400000
COUNT: 500000
COUNT: 600000
COUNT: 700000
COUNT: 800000
COUNT: 900000
COUNT: 1000000
COUNT: 1100000
COUNT: 1200000
COUNT: 1300000
COUNT: 1400000
COUNT: 1500000
COUNT: 1600000
COUNT: 1700000
COUNT: 1800000
COUNT: 1900000
COUNT: 2000000
COUNT: 2100000
COUNT: 2200000
COUNT: 2300000
COUNT: 2400000
COUNT: 2500000
COUNT: 2600000
COUNT: 2700000
COUNT: 2800000
COUNT: 2900000
COUNT: 3000000
COUNT: 3100000
COUNT: 3200000
COUNT: 3300000
COUNT: 3400000
COUNT: 3500000
COUNT: 3600000
COUNT: 3700000
COUNT: 3800000
COUNT: 3900000
COUNT: 4000000
COUNT: 4100000
COUNT: 4200000
COUNT: 4300000
COUNT: 4400000
COUNT: 4500000
COUNT: 4600000
COUNT: 4700000
COUNT: 4800000
COUNT: 4900000
COUNT: 5000000
COUNT: 5100000
COUNT: 5200000
COUNT: 5300000
COUNT: 5400000
COUNT: 5500000
COUNT: 5600000
COUNT: 5700000
COUNT: 5800000
COUNT: 5900000
COUNT: 6000000
COUNT: 6100000
COUNT: 6200000
COUNT: 630000

In [ ]:
#10.15
bbound_nosimilar_multicopies(x6, y, lamb=0.0035, prior_metric="bound", MAXDEPTH = 5)

nrule: 6
ndata: 6907
the rank of x's columns:  [5, 0, 4, 1, 2, 3]
COUNT: 100000
COUNT: 200000
COUNT: 300000
COUNT: 400000
COUNT: 500000
COUNT: 600000
COUNT: 700000
COUNT: 800000
COUNT: 900000
COUNT: 1000000
COUNT: 1100000
COUNT: 1200000
COUNT: 1300000
COUNT: 1400000
COUNT: 1500000
COUNT: 1600000
COUNT: 1700000
COUNT: 1800000
COUNT: 1900000
COUNT: 2000000
COUNT: 2100000
COUNT: 2200000
COUNT: 2300000
COUNT: 2400000
COUNT: 2500000
COUNT: 2600000
COUNT: 2700000
COUNT: 2800000
COUNT: 2900000
COUNT: 3000000
COUNT: 3100000
COUNT: 3200000
COUNT: 3300000
COUNT: 3400000
COUNT: 3500000
COUNT: 3600000
COUNT: 3700000
COUNT: 3800000
COUNT: 3900000
COUNT: 4000000
COUNT: 4100000
COUNT: 4200000
COUNT: 4300000
COUNT: 4400000
COUNT: 4500000
COUNT: 4600000
COUNT: 4700000
COUNT: 4800000
COUNT: 4900000
COUNT: 5000000
COUNT: 5100000
COUNT: 5200000
COUNT: 5300000
COUNT: 5400000
COUNT: 5500000
COUNT: 5600000
COUNT: 5700000
COUNT: 5800000
COUNT: 5900000
COUNT: 6000000
COUNT: 6100000
COUNT: 6200000
COUNT: 630000

In [ ]:
bbound_nosimilar_multicopies0(x6, y, lamb=0.0035, prior_metric="bound", MAXDEPTH = 5)

nrule: 6
ndata: 6907
the rank of x's columns:  [5, 0, 4, 1, 2, 3]
COUNT: 100000
COUNT: 200000
COUNT: 300000
COUNT: 400000
COUNT: 500000
COUNT: 600000
COUNT: 700000
COUNT: 800000
COUNT: 900000
COUNT: 1000000
COUNT: 1100000
COUNT: 1200000
COUNT: 1300000
COUNT: 1400000
COUNT: 1500000
COUNT: 1600000
COUNT: 1700000
COUNT: 1800000
COUNT: 1900000
COUNT: 2000000
COUNT: 2100000
COUNT: 2200000
COUNT: 2300000
COUNT: 2400000
COUNT: 2500000
COUNT: 2600000
COUNT: 2700000
COUNT: 2800000
COUNT: 2900000
COUNT: 3000000
COUNT: 3100000
COUNT: 3200000
COUNT: 3300000
COUNT: 3400000
COUNT: 3500000
COUNT: 3600000
COUNT: 3700000
COUNT: 3800000
COUNT: 3900000
COUNT: 4000000
COUNT: 4100000
COUNT: 4200000
COUNT: 4300000
COUNT: 4400000
COUNT: 4500000
COUNT: 4600000
COUNT: 4700000
COUNT: 4800000
COUNT: 4900000
COUNT: 5000000
COUNT: 5100000
COUNT: 5200000
COUNT: 5300000
COUNT: 5400000
COUNT: 5500000
COUNT: 5600000
COUNT: 5700000
COUNT: 5800000
COUNT: 5900000
COUNT: 6000000
COUNT: 6100000
COUNT: 6200000
COUNT: 630000

In [19]:
#10.15
bbound_nosimilar_multicopies(x, y, lamb=0.0035, prior_metric="bound", MAXDEPTH = 5, logon=True)

nrule: 5
ndata: 6907
the rank of x's columns:  [4, 0, 1, 2, 3]
====== New Tree Duplicated!!! ======
sorted_new_tree_rules: ((-2, -1), (-2, 1), (-1, 2), (1, 2))
====== New Tree Duplicated!!! ======
sorted_new_tree_rules: ((-5, -2), (-5, 2), (-2, 5), (2, 5))
====== New Tree Duplicated!!! ======
sorted_new_tree_rules: ((-4, -1), (-4, 1), (-1, 4), (1, 4))
====== New Tree Duplicated!!! ======
sorted_new_tree_rules: ((-4, -2), (-4, 2), (-2, 4), (2, 4))
====== New Tree Duplicated!!! ======
sorted_new_tree_rules: ((-5, -4), (-5, 4), (-4, 5), (4, 5))
====== New Tree Duplicated!!! ======
sorted_new_tree_rules: ((-3, -1), (-3, 1), (-1, 3), (1, 3))
====== New Tree Duplicated!!! ======
sorted_new_tree_rules: ((-3, -2), (-3, 2), (-2, 3), (2, 3))
====== New Tree Duplicated!!! ======
sorted_new_tree_rules: ((-5, -3), (-5, 3), (-3, 5), (3, 5))
====== New Tree Duplicated!!! ======
sorted_new_tree_rules: ((-4, -3, -1), (-4, -3, 1), (-3, -1, 4), (-3, 1, 4), (-2, -1, 3), (-1, 2, 3), (1, 3))
====== New Tree

====== New Tree Duplicated!!! ======
sorted_new_tree_rules: ((-5, -3, -2, -1), (-5, -3, -1, 2), (-5, -2, -1, 3), (-5, -2, 1, 3), (-4, -3, -2, 1), (-4, -3, 1, 2), (-3, -2, -1, 5), (-3, -1, 2, 5), (-3, 1, 4), (-2, 3, 5), (2, 3))
====== New Tree Duplicated!!! ======
sorted_new_tree_rules: ((-5, -2, -1, 3), (-5, -2, 1, 3), (-4, -3, -2, -1), (-4, -3, -2, 1), (-4, -3, -1, 2), (-4, -3, 1, 2), (-3, -2, -1, 4), (-3, -1, 2, 4), (-3, 1, 4), (-2, 3, 5), (2, 3))
====== New Tree Duplicated!!! ======
sorted_new_tree_rules: ((-5, -4, -3, -1), (-5, -3, -1, 4), (-5, -2, -1, 3), (-5, -2, 1, 3), (-4, -3, -2, 1), (-4, -3, -1, 5), (-4, -3, 1, 2), (-3, -1, 4, 5), (-3, 1, 4), (-2, 3, 5), (2, 3))
====== New Tree Duplicated!!! ======
sorted_new_tree_rules: ((-5, -4, -3, -1), (-5, -3, -1, 4), (-5, -2, -1, 3), (-5, -1, 2, 3), (-4, -3, -2, 1), (-4, -3, -1, 5), (-3, -2, 1, 4), (-3, -1, 4, 5), (-3, 1, 2), (-1, 3, 5), (1, 3))
====== New Tree Duplicated!!! ======
sorted_new_tree_rules: ((-5, -2, -1, 3), (-5, -1, 2, 3)

sorted_new_tree_rules: ((-5, -4, -3, -2), (-5, -4, -2, 3), (-5, -4, -1, 2), (-5, -2, -1, 4), (-5, -2, 1, 4), (-4, -3, -2, 5), (-4, -2, 3, 5), (-4, -1, 2, 5), (-4, 1, 2), (-2, 4, 5), (2, 4))
====== New Tree Duplicated!!! ======
sorted_new_tree_rules: ((-5, -2, -1, 4), (-5, -2, 1, 4), (-4, -3, -2, -1), (-4, -3, -2, 1), (-4, -3, -1, 2), (-4, -2, -1, 3), (-4, -2, 1, 3), (-4, -1, 2, 3), (-4, 1, 2), (-2, 4, 5), (2, 4))
====== New Tree Duplicated!!! ======
sorted_new_tree_rules: ((-5, -4, -1, 2), (-5, -2, -1, 4), (-5, -2, 1, 4), (-4, -3, -2, -1), (-4, -3, -2, 1), (-4, -2, -1, 3), (-4, -2, 1, 3), (-4, -1, 2, 5), (-4, 1, 2), (-2, 4, 5), (2, 4))
====== New Tree Duplicated!!! ======
sorted_new_tree_rules: ((-5, -4, -2, -1), (-5, -4, -1, 2), (-5, -2, -1, 4), (-5, -2, 1, 4), (-4, -3, -2, 1), (-4, -2, -1, 5), (-4, -2, 1, 3), (-4, -1, 2, 5), (-4, 1, 2), (-2, 4, 5), (2, 4))
====== New Tree Duplicated!!! ======
sorted_new_tree_rules: ((-5, -4, -3, -2), (-5, -3, -2, 4), (-5, -2, -1, 3), (-5, -2, 1, 3), 

====== New Tree Duplicated!!! ======
sorted_new_tree_rules: ((-5, -4, -2, -1), (-5, -4, -1, 2), (-5, -3, -2, 1), (-5, -2, -1, 4), (-5, -2, 1, 3), (-5, -1, 2, 4), (-5, 1, 2), (-4, -3, -2, 5), (-4, -2, 3, 5), (-4, 2, 5), (4, 5))
====== New Tree Duplicated!!! ======
sorted_new_tree_rules: ((-5, -4, -2, -1), (-5, -4, -2, 1), (-5, -4, -1, 2), (-5, -2, -1, 4), (-5, -2, 1, 4), (-5, -1, 2, 4), (-5, 1, 2), (-4, -3, -2, 5), (-4, -2, 3, 5), (-4, 2, 5), (4, 5))
====== New Tree Duplicated!!! ======
sorted_new_tree_rules: ((-5, -4, -3, -2), (-5, -4, -2, 3), (-5, -3, -1, 2), (-5, -2, -1, 4), (-5, -2, 1, 4), (-5, -1, 2, 3), (-4, -3, -2, 5), (-4, -2, 3, 5), (-2, 4, 5), (-1, 2, 5), (1, 2))
====== New Tree Duplicated!!! ======
sorted_new_tree_rules: ((-5, -4, -3, -2), (-5, -4, -2, 3), (-5, -4, -1, 2), (-5, -2, -1, 4), (-5, -2, 1, 4), (-5, -1, 2, 4), (-4, -3, -2, 5), (-4, -2, 3, 5), (-2, 4, 5), (-1, 2, 5), (1, 2))
====== New Tree Duplicated!!! ======
sorted_new_tree_rules: ((-5, -3, -1, 2), (-5, -2, -1, 4

====== New Tree Duplicated!!! ======
sorted_new_tree_rules: ((-5, -2, -1, 3), (-5, -1, 2, 3), (-5, 1, 3), (-4, -3, -2, -1), (-4, -3, -2, 1), (-4, -3, 2), (-3, -2, -1, 4), (-3, -2, 1, 4), (-3, 2, 4), (3, 5))
====== New Tree Duplicated!!! ======
sorted_new_tree_rules: ((-5, -4, -3, -2), (-5, -3, -2, 4), (-5, -2, -1, 3), (-5, -1, 2, 3), (-5, 1, 3), (-4, -3, -2, 5), (-4, -3, 2), (-3, -2, 4, 5), (-3, 2, 4), (3, 5))
====== New Tree Duplicated!!! ======
sorted_new_tree_rules: ((-5, -2, -1, 3), (-5, -2, 1, 3), (-5, 2, 3), (-4, -3, -2, -1), (-4, -3, -2, 1), (-3, -2, -1, 4), (-3, -2, 1, 4), (-3, 2), (3, 5))
====== New Tree Duplicated!!! ======
sorted_new_tree_rules: ((-5, -2, -1, 3), (-5, -2, 1, 3), (-4, -3, -2, -1), (-4, -3, -2, 1), (-3, -2, -1, 4), (-3, -2, 1, 4), (-3, 2), (-2, 3, 5), (2, 3))
====== New Tree Duplicated!!! ======
sorted_new_tree_rules: ((-5, -2, -1, 4), (-5, -2, 1, 4), (-4, -3, -2, -1), (-4, -3, -2, 1), (-4, -2, -1, 3), (-4, -2, 1, 3), (-3, 2), (-2, 4, 5), (2, 3))
====== New Tr

sorted_new_tree_rules: ((-5, -4, -3, 1), (-5, -4, -2, -1), (-5, -4, -1, 2), (-5, -4, 1, 3), (-5, -2, -1, 4), (-5, -1, 2, 4), (-5, 1, 4), (-3, 5), (3, 5))
====== New Tree Duplicated!!! ======
sorted_new_tree_rules: ((-5, -4, -2, -1), (-5, -4, -1, 2), (-5, -3, -2, 1), (-5, -3, 1, 2), (-5, -2, -1, 4), (-5, -1, 2, 4), (-5, 1, 3), (-3, -2, 5), (-3, 2, 5), (3, 5))
====== New Tree Duplicated!!! ======
sorted_new_tree_rules: ((-5, -4, -3, 1), (-5, -4, -2, -1), (-5, -4, -1, 2), (-5, -3, 1, 4), (-5, -2, -1, 4), (-5, -1, 2, 4), (-5, 1, 3), (-3, -2, 5), (-3, 2, 5), (3, 5))
====== New Tree Duplicated!!! ======
sorted_new_tree_rules: ((-5, -1, 2), (-5, 1, 2), (-4, -3, -2, -1), (-4, -3, -2, 1), (-3, -2, -1, 4), (-3, -2, 1, 4), (-2, 3), (2, 5))
====== New Tree Duplicated!!! ======
sorted_new_tree_rules: ((-5, -1, 2), (-5, 1, 2), (-4, -3, -2, -1), (-4, -3, -2, 1), (-4, -2, -1, 3), (-4, -2, 1, 3), (-2, 4), (2, 5))
====== New Tree Duplicated!!! ======
sorted_new_tree_rules: ((-5, -4, -2, 1), (-5, -4, 1, 

====== New Tree Duplicated!!! ======
sorted_new_tree_rules: ((-5, -4, -2, -1), (-5, -4, -1, 2), (-5, -2, -1, 4), (-5, -1, 2, 4), (-4, -2, -1, 5), (-2, -1, 4, 5), (-1, 2, 5), (1,))
====== New Tree Duplicated!!! ======
sorted_new_tree_rules: ((-5, -4, -3, -1), (-5, -3, -1, 4), (-5, -2, -1, 3), (-5, -2, 1, 3), (-4, -3, -1, 5), (-3, -2, 1), (-3, -1, 4, 5), (-3, 1, 2), (-2, 3, 5), (2, 3))
====== New Tree Duplicated!!! ======
sorted_new_tree_rules: ((-5, -3, -2, -1), (-5, -3, -1, 2), (-5, -2, -1, 3), (-5, -2, 1, 3), (-4, -3, 1), (-3, -2, -1, 5), (-3, -1, 2, 5), (-3, 1, 4), (-2, 3, 5), (2, 3))
====== New Tree Duplicated!!! ======
sorted_new_tree_rules: ((-5, -2, -1, 3), (-5, -2, 1, 3), (-4, -3, -2, -1), (-4, -3, -1, 2), (-4, -3, 1), (-3, -2, -1, 4), (-3, -1, 2, 4), (-3, 1, 4), (-2, 3, 5), (2, 3))
====== New Tree Duplicated!!! ======
sorted_new_tree_rules: ((-5, -4, -3, -1), (-5, -3, -1, 4), (-5, -2, -1, 3), (-5, -2, 1, 3), (-4, -3, -1, 5), (-4, -3, 1), (-3, -1, 4, 5), (-3, 1, 4), (-2, 3, 5), 

====== New Tree Duplicated!!! ======
sorted_new_tree_rules: ((-5, -4, -3, -2, -1), (-5, -4, -3, -1, 2), (-5, -4, -2, -1, 3), (-5, -2, -1, 4), (-4, -3, -2, -1, 5), (-4, -3, -2, 1), (-4, -3, -1, 2, 5), (-4, -3, 1, 2), (-4, -2, -1, 3, 5), (-4, -1, 2, 3), (-3, 1, 4), (-2, -1, 4, 5), (-1, 2, 4), (1, 3))
====== New Tree Duplicated!!! ======
sorted_new_tree_rules: ((-5, -4, -3, -2, -1), (-5, -4, -3, -1, 2), (-5, -4, -2, -1, 3), (-5, -2, -1, 4), (-4, -3, -2, -1, 5), (-4, -3, -2, 1), (-4, -3, -1, 2, 5), (-4, -3, 1, 2), (-4, -2, -1, 3, 5), (-4, 1, 3), (-3, -1, 2, 4), (-2, -1, 4, 5), (-1, 2, 3), (1, 4))
====== New Tree Duplicated!!! ======
sorted_new_tree_rules: ((-5, -4, -3, -2, -1), (-5, -4, -3, -1, 2), (-5, -4, -2, -1, 3), (-5, -2, -1, 4), (-4, -3, -2, -1, 5), (-4, -3, -2, 1), (-4, -3, -1, 2, 5), (-4, -3, 1, 2), (-4, -2, -1, 3, 5), (-4, -1, 2, 3), (-4, 1, 3), (-2, -1, 4, 5), (-1, 2, 4), (1, 4))
====== New Tree Duplicated!!! ======
sorted_new_tree_rules: ((-5, -4, -3, -2, -1), (-5, -4, -3, -1, 

====== New Tree Duplicated!!! ======
sorted_new_tree_rules: ((-5, -4, -3, -2, -1), (-5, -4, -3, -1, 2), (-5, -3, -2, -1, 4), (-5, -3, -1, 2, 4), (-5, -1, 3), (-4, -3, -2, -1, 5), (-4, -3, -2, 1), (-4, -3, 1, 2), (-3, -2, -1, 4, 5), (-3, -1, 2, 5), (-3, 1, 4), (-1, 3, 5), (1, 3))
====== New Tree Duplicated!!! ======
sorted_new_tree_rules: ((-5, -4, -3, -2, -1), (-5, -4, -3, -1, 2), (-5, -3, -2, -1, 4), (-5, -3, -1, 2, 4), (-5, -2, -1, 3), (-5, -1, 2, 3), (-4, -3, -2, -1, 5), (-4, -3, -2, 1), (-3, -2, -1, 4, 5), (-3, -2, 1, 4), (-3, -1, 2, 5), (-2, 1, 3), (-1, 3, 5), (1, 2))
====== New Tree Duplicated!!! ======
sorted_new_tree_rules: ((-5, -4, -3, -2, -1), (-5, -4, -3, -1, 2), (-5, -3, -2, -1, 4), (-5, -3, -1, 2, 4), (-5, -2, -1, 3), (-5, -1, 2, 3), (-4, -3, -2, -1, 5), (-4, -3, -2, 1), (-3, -2, -1, 4, 5), (-3, -2, 1, 4), (-3, -1, 2, 5), (-2, 1, 3), (-1, 3, 5), (1, 2))
====== New Tree Duplicated!!! ======
sorted_new_tree_rules: ((-5, -4, -3, -2, -1), (-5, -4, -3, -1, 2), (-5, -3, -2, -1,

====== New Tree Duplicated!!! ======
sorted_new_tree_rules: ((-5, -4, -3, -2, -1), (-5, -4, -3, -1, 2), (-5, -4, -2, -1, 3), (-5, -4, -1, 2, 3), (-5, -2, -1, 4), (-5, -2, 1, 4), (-4, -3, -2, 1), (-4, -3, -1, 5), (-4, -3, 1, 2), (-4, -1, 3, 5), (-4, 1, 3), (-2, 4, 5), (2, 4))
====== New Tree Duplicated!!! ======
sorted_new_tree_rules: ((-5, -4, -3, -2, -1), (-5, -4, -3, -1, 2), (-5, -4, -2, -1, 3), (-5, -4, -1, 2, 3), (-5, -2, -1, 4), (-4, -3, -2, 1), (-4, -3, -1, 5), (-4, -3, 1, 2), (-4, -1, 3, 5), (-4, 1, 3), (-2, -1, 4, 5), (-2, 1, 4), (2, 4))
>>> log: True
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  8.016226768493652
lambda:  0.0035
leaves:  [(3,), (-3, 1), (-5, -3, -1), (-4, -3, -1, 5), (-3, -1, 4, 5)]
prediction:  [1, 1, 0, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  22194
time when the best tree is achieved:  7.841000080108643
TOTAL COUNT:  22211


In [26]:
#10.15
bbound_nosimilar_multicopies(x, y, lamb=0.0035, prior_metric="curiosity", MAXDEPTH = 5, logon=True)

nrule: 5
ndata: 6907
the rank of x's columns:  [4, 0, 1, 2, 3]
>>> log: True
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  5.991255760192871
lambda:  0.0035
leaves:  [(1,), (-4, -3, -1), (-4, -1, 3), (-5, -1, 4), (-1, 4, 5)]
prediction:  [1, 0, 1, 0, 1]
Objective:  0.35411502823222823
COUNT of the best tree:  137
time when the best tree is achieved:  1.634636640548706
TOTAL COUNT:  7172


In [22]:
#10.15
bbound_nosimilar_multicopies(x, y, lamb=0.0035, prior_metric="bound", MAXDEPTH = 5)

nrule: 5
ndata: 6907
the rank of x's columns:  [4, 0, 1, 2, 3]
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  7.0357537269592285
lambda:  0.0035
leaves:  [(3,), (-3, 1), (-5, -3, -1), (-4, -3, -1, 5), (-3, -1, 4, 5)]
prediction:  [1, 1, 0, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  22194
time when the best tree is achieved:  6.721847295761108
TOTAL COUNT:  22338


In [19]:
#10.15
bbound_nosimilar_multicopies(x, y, lamb=0.0035, prior_metric="bound", MAXDEPTH = 5)

nrule: 5
ndata: 6907
the rank of x's columns:  [4, 0, 1, 2, 3]
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  7.0545947551727295
lambda:  0.0035
leaves:  [(3,), (-3, 1), (-5, -3, -1), (-4, -3, -1, 5), (-3, -1, 4, 5)]
prediction:  [1, 1, 0, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  22194
time when the best tree is achieved:  6.9284446239471436
TOTAL COUNT:  22211


In [23]:
#10.15
bbound_nosimilar_multicopies(x, y, lamb=0.0035, prior_metric="curiosity", MAXDEPTH = 5)

nrule: 5
ndata: 6907
the rank of x's columns:  [4, 0, 1, 2, 3]
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  6.0564515590667725
lambda:  0.0035
leaves:  [(1,), (-4, -3, -1), (-4, -1, 3), (-5, -1, 4), (-1, 4, 5)]
prediction:  [1, 0, 1, 0, 1]
Objective:  0.35411502823222823
COUNT of the best tree:  137
time when the best tree is achieved:  1.7191762924194336
TOTAL COUNT:  7172


In [24]:
#10.15
bbound_nosimilar_multicopies(x, y, lamb=0.0035, prior_metric="gini", MAXDEPTH = 5)

nrule: 5
ndata: 6907
the rank of x's columns:  [4, 0, 1, 2, 3]
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  6.219820737838745
lambda:  0.0035
leaves:  [(1,), (-1, 3), (-5, -3, -1), (-4, -3, -1, 5), (-3, -1, 4, 5)]
prediction:  [1, 1, 0, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  6439
time when the best tree is achieved:  5.406334161758423
TOTAL COUNT:  9138


In [79]:
bbound_nosimilar_multicopies(x, y, lamb=0.0035, prior_metric="bound", MAXDEPTH = 4)

nrule: 5
ndata: 6907
the rank of x's columns:  [4, 0, 1, 2, 3]
COUNT: 100000
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  9.46976089477539
lambda:  0.0035
leaves:  [(3,), (-5, -3, -1), (-5, -3, 1), (-4, -3, 5), (-3, 4, 5)]
prediction:  [1, 0, 1, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  2309
time when the best tree is achieved:  3.1118435859680176
TOTAL COUNT:  167670


In [21]:
#class Tree
bbound_nosimilar_multicopies(x, y, lamb=0.0035, prior_metric="bound", MAXDEPTH = 4)

nrule: 5
ndata: 6907
the rank of x's columns:  [4, 0, 1, 2, 3]
COUNT: 100000
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  8.917646169662476
lambda:  0.0035
leaves:  [(3,), (-5, -3, -1), (-5, -3, 1), (-4, -3, 5), (-3, 4, 5)]
prediction:  [1, 0, 1, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  2309
time when the best tree is achieved:  3.0205655097961426
TOTAL COUNT:  167670


In [19]:
#2*tree_new.metric - R_c
bbound_nosimilar_multicopies(x, y, lamb=0.0035, prior_metric="bound", MAXDEPTH = 4)

nrule: 5
ndata: 6907
the rank of x's columns:  [4, 0, 1, 2, 3]
COUNT: 100000
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  9.045903444290161
lambda:  0.0035
leaves:  [(3,), (-5, -3, -1), (-5, -3, 1), (-4, -3, 5), (-3, 4, 5)]
prediction:  [1, 0, 1, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  2309
time when the best tree is achieved:  3.013650894165039
TOTAL COUNT:  166788


In [80]:
bbound_nosimilar_multicopies(x, y, lamb=0.0035, prior_metric="bound", MAXDEPTH = 5)

nrule: 5
ndata: 6907
the rank of x's columns:  [4, 0, 1, 2, 3]
COUNT: 100000
COUNT: 200000
COUNT: 300000
COUNT: 400000
COUNT: 500000
COUNT: 600000
COUNT: 700000
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  31.859504461288452
lambda:  0.0035
leaves:  [(3,), (-5, -3, -1), (-5, -3, 1), (-4, -3, 5), (-3, 4, 5)]
prediction:  [1, 0, 1, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  3862
time when the best tree is achieved:  3.1401665210723877
TOTAL COUNT:  773889


In [21]:
#10.15
bbound_nosimilar_multicopies(x, y, lamb=0.0035, prior_metric="objective", MAXDEPTH = 5)

nrule: 5
ndata: 6907
the rank of x's columns:  [4, 0, 1, 2, 3]
====== New Tree Duplicated!!! ======
====== New Tree Duplicated!!! ======
====== New Tree Duplicated!!! ======
====== New Tree Duplicated!!! ======
====== New Tree Duplicated!!! ======
====== New Tree Duplicated!!! ======
====== New Tree Duplicated!!! ======
====== New Tree Duplicated!!! ======
====== New Tree Duplicated!!! ======
====== New Tree Duplicated!!! ======
====== New Tree Duplicated!!! ======
====== New Tree Duplicated!!! ======
====== New Tree Duplicated!!! ======
====== New Tree Duplicated!!! ======
====== New Tree Duplicated!!! ======
====== New Tree Duplicated!!! ======
====== New Tree Duplicated!!! ======
====== New Tree Duplicated!!! ======
====== New Tree Duplicated!!! ======
====== New Tree Duplicated!!! ======
====== New Tree Duplicated!!! ======
====== New Tree Duplicated!!! ======
====== New Tree Duplicated!!! ======
====== New Tree Duplicated!!! ======
====== New Tree Duplicated!!! ======
====== New T

In [28]:
#class Tree
bbound_nosimilar_multicopies(x, y, lamb=0.0035, prior_metric="objective", MAXDEPTH = 5)

nrule: 5
ndata: 6907
the rank of x's columns:  [4, 0, 1, 2, 3]
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  4.899874210357666
lambda:  0.0035
leaves:  [(3,), (-3, 1), (-5, -3, -1), (-4, -3, -1, 5), (-3, -1, 4, 5)]
prediction:  [1, 1, 0, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  18868
time when the best tree is achieved:  3.274327039718628
TOTAL COUNT:  42628


In [20]:
#2*tree_new.metric - R_c
bbound_nosimilar_multicopies(x, y, lamb=0.0035, prior_metric="bound", MAXDEPTH = 5)

nrule: 5
ndata: 6907
the rank of x's columns:  [4, 0, 1, 2, 3]
COUNT: 100000
COUNT: 200000
COUNT: 300000
COUNT: 400000
COUNT: 500000
COUNT: 600000
COUNT: 700000
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  31.00615119934082
lambda:  0.0035
leaves:  [(3,), (-5, -3, -1), (-5, -3, 1), (-4, -3, 5), (-3, 4, 5)]
prediction:  [1, 0, 1, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  3862
time when the best tree is achieved:  3.1813623905181885
TOTAL COUNT:  771830


In [20]:
#2*tree_new.metric - R_c
bbound_nosimilar(x, y, lamb=0.0035, prior_metric="bound", MAXDEPTH = 5)

nrule: 5
ndata: 6907
the rank of x's columns:  [4, 0, 1, 2, 3]
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  6.523130893707275
lambda:  0.0035
leaves:  [(3,), (-3, 1), (-4, -3, -1), (-5, -3, -1, 4), (-3, -1, 4, 5)]
prediction:  [1, 1, 0, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  5874
time when the best tree is achieved:  4.709397315979004
TOTAL COUNT:  19662


In [81]:
bbound_nosimilar_multicopies(x6, y, lamb=0.0035, prior_metric="bound", MAXDEPTH = 4)

nrule: 6
ndata: 6907
the rank of x's columns:  [5, 0, 4, 1, 2, 3]
COUNT: 100000
COUNT: 200000
COUNT: 300000
COUNT: 400000
COUNT: 500000
COUNT: 600000
COUNT: 700000
COUNT: 800000
COUNT: 900000
COUNT: 1000000
COUNT: 1100000
COUNT: 1200000
COUNT: 1300000
COUNT: 1400000
COUNT: 1500000
COUNT: 1600000
COUNT: 1700000
COUNT: 1800000
COUNT: 1900000
COUNT: 2000000
COUNT: 2100000
COUNT: 2200000
COUNT: 2300000
COUNT: 2400000
COUNT: 2500000
COUNT: 2600000
COUNT: 2700000
COUNT: 2800000
COUNT: 2900000
COUNT: 3000000
COUNT: 3100000
COUNT: 3200000
COUNT: 3300000
COUNT: 3400000
COUNT: 3500000
COUNT: 3600000
COUNT: 3700000
COUNT: 3800000
COUNT: 3900000
COUNT: 4000000
COUNT: 4100000
COUNT: 4200000
COUNT: 4300000
COUNT: 4400000
COUNT: 4500000
COUNT: 4600000
COUNT: 4700000
COUNT: 4800000
COUNT: 4900000
COUNT: 5000000
COUNT: 5100000
COUNT: 5200000
COUNT: 5300000
COUNT: 5400000
COUNT: 5500000
COUNT: 5600000


KeyboardInterrupt: 

In [ ]:
#2*tree_new.metric - R_c


In [75]:
bbound_nosimilar(x_corr, y, lamb=0.0035, prior_metric="objective", MAXDEPTH = 4)

nrule: 6
ndata: 6907
the rank of x's columns:  [1, 5, 0, 2, 3, 4]
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  15.236387729644775
lambda:  0.0035
leaves:  [(1,), (-1, 6), (-6, -5, -1), (-6, -4, -1, 5), (-6, -1, 4, 5)]
prediction:  [1, 1, 0, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  23402
time when the best tree is achieved:  7.3162524700164795
TOTAL COUNT:  71282


In [78]:
bbound_similar_when_sub(x_corr, y, lamb=0.0035, prior_metric="objective", MAXDEPTH = 4)

nrule: 6
ndata: 6907
the rank of x's columns:  [1, 5, 0, 2, 3, 4]
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  14.7958505153656
lambda:  0.0035
leaves:  [(1,), (-1, 6), (-6, -5, -1), (-6, -4, -1, 5), (-6, -1, 4, 5)]
prediction:  [1, 1, 0, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  21948
time when the best tree is achieved:  7.234368562698364
TOTAL COUNT:  66473


In [17]:
bbound_nosimilar(x, y, lamb=0.0035, prior_metric="objective", MAXDEPTH = 4)

nrule: 5
ndata: 6907
the rank of x's columns:  [4, 0, 1, 2, 3]
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  4.870143890380859
lambda:  0.0035
leaves:  [(3,), (-3, 1), (-5, -3, -1), (-4, -3, -1, 5), (-3, -1, 4, 5)]
prediction:  [1, 1, 0, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  1794
time when the best tree is achieved:  2.987828016281128
TOTAL COUNT:  6864


In [17]:
bbound_similar_when_sub(x, y, lamb=0.0035, prior_metric="objective", MAXDEPTH = 4)

nrule: 5
ndata: 6907
the rank of x's columns:  [4, 0, 1, 2, 3]
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  4.669439077377319
lambda:  0.0035
leaves:  [(3,), (-3, 1), (-5, -3, -1), (-4, -3, -1, 5), (-3, -1, 4, 5)]
prediction:  [1, 1, 0, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  1794
time when the best tree is achieved:  2.798046350479126
TOTAL COUNT:  6864


In [19]:
bbound_nosimilar(x, y, lamb=0.0035, prior_metric="objective", MAXDEPTH = 4)

nrule: 5
ndata: 6907
the rank of x's columns:  [4, 0, 1, 2, 3]
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  4.694583892822266
lambda:  0.0035
leaves:  [(3,), (-3, 1), (-5, -3, -1), (-4, -3, -1, 5), (-3, -1, 4, 5)]
prediction:  [1, 1, 0, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  1794
time when the best tree is achieved:  2.9489943981170654
TOTAL COUNT:  6864


In [18]:
bbound_nosimilar_multicopies(x, y, lamb=0.0035, prior_metric="objective", MAXDEPTH = 4)

nrule: 5
ndata: 6907
the rank of x's columns:  [4, 0, 1, 2, 3]
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  4.414027690887451
lambda:  0.0035
leaves:  [(3,), (-3, 1), (-5, -3, -1), (-4, -3, -1, 5), (-3, -1, 4, 5)]
prediction:  [1, 1, 0, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  9037
time when the best tree is achieved:  3.081894636154175
TOTAL COUNT:  17705


In [17]:
bbound_nosimilar_multicopies(x, y, lamb=0.0035, prior_metric="objective", MAXDEPTH = 4)

nrule: 5
ndata: 6907
the rank of x's columns:  [4, 0, 1, 2, 3]
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  4.299317359924316
lambda:  0.0035
leaves:  [(3,), (-3, 1), (-5, -3, -1), (-4, -3, -1, 5), (-3, -1, 4, 5)]
prediction:  [1, 1, 0, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  9037
time when the best tree is achieved:  3.013827085494995
TOTAL COUNT:  17705


In [17]:
bbound_similar(x, y, lamb=0.0035, prior_metric="objective", MAXDEPTH = 4)

nrule: 5
ndata: 6907
the rank of x's columns:  [4, 0, 1, 2, 3]
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  7.858139753341675
lambda:  0.0035
leaves:  [(3,), (-3, 1), (-5, -3, -1), (-4, -3, -1, 5), (-3, -1, 4, 5)]
prediction:  [1, 1, 0, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  1794
time when the best tree is achieved:  3.0144846439361572
TOTAL COUNT:  6161


In [18]:
bbound_similar(x, y, lamb=0.0035, prior_metric="objective", MAXDEPTH = 4)

nrule: 5
ndata: 6907
the rank of x's columns:  [4, 0, 1, 2, 3]


KeyboardInterrupt: 

In [23]:
bbound_similar_when_big(x, y, lamb=0.0035, prior_metric="objective", MAXDEPTH = 4, SSB_cache_thres=5, SSB_check_thres=6)

nrule: 5
ndata: 6907
the rank of x's columns:  [4, 0, 1, 2, 3]
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  4.631837606430054
lambda:  0.0035
leaves:  [(3,), (-3, 1), (-5, -3, -1), (-4, -3, -1, 5), (-3, -1, 4, 5)]
prediction:  [1, 1, 0, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  1794
time when the best tree is achieved:  2.7678072452545166
TOTAL COUNT:  6618


In [25]:
bbound_similar_when_big(x, y, lamb=0.0035, prior_metric="objective", MAXDEPTH = 4, SSB_cache_thres=5, SSB_check_thres=7)

nrule: 5
ndata: 6907
the rank of x's columns:  [4, 0, 1, 2, 3]
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  5.738712549209595
lambda:  0.0035
leaves:  [(3,), (-3, 1), (-5, -3, -1), (-4, -3, -1, 5), (-3, -1, 4, 5)]
prediction:  [1, 1, 0, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  1794
time when the best tree is achieved:  2.7859995365142822
TOTAL COUNT:  6285


In [19]:
bbound_similar_priority(x, y, lamb=0.0035, prior_metric="objective", MAXDEPTH = 4)

nrule: 5
ndata: 6907
the rank of x's columns:  [4, 0, 1, 2, 3]
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  8.540912389755249
lambda:  0.0035
leaves:  [(3,), (-3, 1), (-5, -3, -1), (-4, -3, -1, 5), (-3, -1, 4, 5)]
prediction:  [1, 1, 0, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  1794
time when the best tree is achieved:  3.0274953842163086
TOTAL COUNT:  6161


In [19]:
##HERE
bbound_nosimilar(x, y, lamb=0.0035, prior_metric="objective", MAXDEPTH = 5)

nrule: 5
ndata: 6907
the rank of x's columns:  [4, 0, 1, 2, 3]
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  6.402764320373535
lambda:  0.0035
leaves:  [(3,), (-3, 1), (-5, -3, -1), (-4, -3, -1, 5), (-3, -1, 4, 5)]
prediction:  [1, 1, 0, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  5525
time when the best tree is achieved:  3.5685641765594482
TOTAL COUNT:  18855


In [18]:
##HERE
bbound_similar_when_sub(x, y, lamb=0.0035, prior_metric="objective", MAXDEPTH = 5)

nrule: 5
ndata: 6907
the rank of x's columns:  [4, 0, 1, 2, 3]
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  6.263530969619751
lambda:  0.0035
leaves:  [(3,), (-3, 1), (-5, -3, -1), (-4, -3, -1, 5), (-3, -1, 4, 5)]
prediction:  [1, 1, 0, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  5525
time when the best tree is achieved:  3.420346975326538
TOTAL COUNT:  18855


In [17]:
##HERE
bbound_nosimilar(x, y, lamb=0.0035, prior_metric="objective", MAXDEPTH = 5)

nrule: 5
ndata: 6907
the rank of x's columns:  [4, 0, 1, 2, 3]
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  6.3049023151397705
lambda:  0.0035
leaves:  [(3,), (-3, 1), (-5, -3, -1), (-4, -3, -1, 5), (-3, -1, 4, 5)]
prediction:  [1, 1, 0, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  5525
time when the best tree is achieved:  3.5752158164978027
TOTAL COUNT:  18855


In [22]:
bbound_nosimilar_multicopies(x, y, lamb=0.0035, prior_metric="objective", MAXDEPTH = 5)

nrule: 5
ndata: 6907
the rank of x's columns:  [4, 0, 1, 2, 3]
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  4.960108757019043
lambda:  0.0035
leaves:  [(3,), (-3, 1), (-5, -3, -1), (-4, -3, -1, 5), (-3, -1, 4, 5)]
prediction:  [1, 1, 0, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  26561
time when the best tree is achieved:  3.650310754776001
TOTAL COUNT:  47780


In [18]:
bbound_nosimilar_multicopies(x, y, lamb=0.0035, prior_metric="objective", MAXDEPTH = 5)

nrule: 5
ndata: 6907
the rank of x's columns:  [4, 0, 1, 2, 3]
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  4.878471851348877
lambda:  0.0035
leaves:  [(3,), (-3, 1), (-5, -3, -1), (-4, -3, -1, 5), (-3, -1, 4, 5)]
prediction:  [1, 1, 0, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  26561
time when the best tree is achieved:  3.568647623062134
TOTAL COUNT:  47780


In [27]:
bbound_nosimilar(x, y, lamb=0.0035, prior_metric="bound", MAXDEPTH = 5)

nrule: 5
ndata: 6907
the rank of x's columns:  [4, 0, 1, 2, 3]
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  6.355633497238159
lambda:  0.0035
leaves:  [(3,), (-3, 1), (-4, -3, -1), (-5, -3, -1, 4), (-3, -1, 4, 5)]
prediction:  [1, 1, 0, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  5598
time when the best tree is achieved:  4.699038743972778
TOTAL COUNT:  17714


In [21]:
bbound_nosimilar_multicopies(x, y, lamb=0.0035, prior_metric="bound", MAXDEPTH = 5)

nrule: 5
ndata: 6907
the rank of x's columns:  [4, 0, 1, 2, 3]
COUNT: 100000
COUNT: 200000
COUNT: 300000
COUNT: 400000
COUNT: 500000
COUNT: 600000
COUNT: 700000
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  31.16526222229004
lambda:  0.0035
leaves:  [(3,), (-5, -3, -1), (-5, -3, 1), (-4, -3, 5), (-3, 4, 5)]
prediction:  [1, 0, 1, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  3862
time when the best tree is achieved:  3.397043228149414
TOTAL COUNT:  773889


In [19]:
bbound_nosimilar_multicopies(x, y, lamb=0.0035, prior_metric="bound", MAXDEPTH = 5)

nrule: 5
ndata: 6907
the rank of x's columns:  [4, 0, 1, 2, 3]
COUNT: 100000
COUNT: 200000
COUNT: 300000
COUNT: 400000
COUNT: 500000
COUNT: 600000
COUNT: 700000
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  30.564636945724487
lambda:  0.0035
leaves:  [(3,), (-5, -3, -1), (-5, -3, 1), (-4, -3, 5), (-3, 4, 5)]
prediction:  [1, 0, 1, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  3862
time when the best tree is achieved:  3.1217753887176514
TOTAL COUNT:  773889


In [23]:
bbound_nosimilar(x, y, lamb=0.0035, prior_metric="curiosity", MAXDEPTH = 5)

nrule: 5
ndata: 6907
the rank of x's columns:  [4, 0, 1, 2, 3]
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  5.796830654144287
lambda:  0.0035
leaves:  [(3,), (-5, -3, -1), (-5, -3, 1), (-4, -3, 5), (-3, 4, 5)]
prediction:  [1, 0, 1, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  7136
time when the best tree is achieved:  5.222828388214111
TOTAL COUNT:  10672


In [24]:
bbound_nosimilar_multicopies(x, y, lamb=0.0035, prior_metric="curiosity", MAXDEPTH = 5)

nrule: 5
ndata: 6907
the rank of x's columns:  [4, 0, 1, 2, 3]
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  6.391690015792847
lambda:  0.0035
leaves:  [(3,), (-4, -3, 5), (-3, 4, 5), (-5, -3, -1), (-5, -3, 1)]
prediction:  [1, 0, 1, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  46334
time when the best tree is achieved:  5.025545597076416
TOTAL COUNT:  86712


In [25]:
bbound_nosimilar(x, y, lamb=0.0035, prior_metric="gini", MAXDEPTH = 5)

nrule: 5
ndata: 6907
the rank of x's columns:  [4, 0, 1, 2, 3]
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  6.170199155807495
lambda:  0.0035
leaves:  [(1,), (-1, 2), (-2, -1, 3), (-5, -3, -2, -1), (-4, -3, -2, -1, 5), (-3, -2, -1, 4, 5)]
prediction:  [1, 0, 1, 0, 0, 1]
Objective:  0.3560224410018822
COUNT of the best tree:  1059
time when the best tree is achieved:  2.762575149536133
TOTAL COUNT:  13679


In [26]:
bbound_nosimilar_multicopies(x, y, lamb=0.0035, prior_metric="gini", MAXDEPTH = 5)

nrule: 5
ndata: 6907
the rank of x's columns:  [4, 0, 1, 2, 3]
COUNT: 100000
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  7.997255563735962
lambda:  0.0035
leaves:  [(1,), (-1, 2), (-2, -1, 3), (-5, -3, -2, -1), (-4, -3, -2, -1, 5), (-3, -2, -1, 4, 5)]
prediction:  [1, 0, 1, 0, 0, 1]
Objective:  0.35602244100188213
COUNT of the best tree:  44
time when the best tree is achieved:  0.7691073417663574
TOTAL COUNT:  112619


In [27]:
bbound_nosimilar(x, y, lamb=0.0035, prior_metric="entropy", MAXDEPTH = 5)

nrule: 5
ndata: 6907
the rank of x's columns:  [4, 0, 1, 2, 3]
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  6.236457109451294
lambda:  0.0035
leaves:  [(1,), (-1, 2), (-2, -1, 3), (-5, -3, -2, -1), (-4, -3, -2, -1, 5), (-3, -2, -1, 4, 5)]
prediction:  [1, 0, 1, 0, 0, 1]
Objective:  0.3560224410018822
COUNT of the best tree:  1053
time when the best tree is achieved:  2.7753746509552
TOTAL COUNT:  13675


In [28]:
bbound_nosimilar_multicopies(x, y, lamb=0.0035, prior_metric="entropy", MAXDEPTH = 5)

nrule: 5
ndata: 6907
the rank of x's columns:  [4, 0, 1, 2, 3]
COUNT: 100000
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  9.00658655166626
lambda:  0.0035
leaves:  [(1,), (-1, 2), (-2, -1, 3), (-5, -3, -2, -1), (-4, -3, -2, -1, 5), (-3, -2, -1, 4, 5)]
prediction:  [1, 0, 1, 0, 0, 1]
Objective:  0.35602244100188213
COUNT of the best tree:  44
time when the best tree is achieved:  0.7742917537689209
TOTAL COUNT:  112611


In [21]:
bbound_similar(x, y, lamb=0.0035, prior_metric="objective", MAXDEPTH = 5)

nrule: 5
ndata: 6907
the rank of x's columns:  [4, 0, 1, 2, 3]
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  54.60772776603699
lambda:  0.0035
leaves:  [(3,), (-3, 1), (-5, -3, -1), (-4, -3, -1, 5), (-3, -1, 4, 5)]
prediction:  [1, 1, 0, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  5525
time when the best tree is achieved:  5.776895523071289
TOTAL COUNT:  17633


In [26]:
bbound_similar_when_big(x, y, lamb=0.0035, prior_metric="objective", MAXDEPTH = 5, SSB_cache_thres=5, SSB_check_thres=6)

nrule: 5
ndata: 6907
the rank of x's columns:  [4, 0, 1, 2, 3]
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  7.61355996131897
lambda:  0.0035
leaves:  [(3,), (-3, 1), (-5, -3, -1), (-4, -3, -1, 5), (-3, -1, 4, 5)]
prediction:  [1, 1, 0, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  5525
time when the best tree is achieved:  3.545598030090332
TOTAL COUNT:  18693


In [48]:
bbound_similar_when_big(x, y, lamb=0.0035, prior_metric="objective", MAXDEPTH = 5, SSB_cache_thres=7, SSB_check_thres=5)

nrule: 5
ndata: 6907
the rank of x's columns:  [4, 0, 1, 2, 3]
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  7.005394697189331
lambda:  0.0035
leaves:  [(3,), (-3, 1), (-5, -3, -1), (-4, -3, -1, 5), (-3, -1, 4, 5)]
prediction:  [1, 1, 0, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  5525
time when the best tree is achieved:  3.521812915802002
TOTAL COUNT:  18734


In [23]:
bbound_similar_priority(x, y, lamb=0.0035, prior_metric="objective", MAXDEPTH = 5)

nrule: 5
ndata: 6907
the rank of x's columns:  [4, 0, 1, 2, 3]
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  65.0609200000763
lambda:  0.0035
leaves:  [(3,), (-3, 1), (-5, -3, -1), (-4, -3, -1, 5), (-3, -1, 4, 5)]
prediction:  [1, 1, 0, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  5525
time when the best tree is achieved:  6.38911247253418
TOTAL COUNT:  17633


In [22]:
bbound_nosimilar(x6, y, lamb=0.0035, prior_metric="objective", MAXDEPTH = 4)

nrule: 6
ndata: 6907
the rank of x's columns:  [5, 0, 4, 1, 2, 3]
COUNT: 100000
COUNT: 200000
COUNT: 300000
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  41.53455448150635
lambda:  0.0035
leaves:  [(4,), (-4, 1), (-4, -3, -1), (-5, -4, -1, 3), (-4, -1, 3, 5)]
prediction:  [1, 1, 0, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  186494
time when the best tree is achieved:  25.435484647750854
TOTAL COUNT:  346501


In [20]:
bbound_nosimilar(x6, y, lamb=0.0035, prior_metric="objective", MAXDEPTH = 4)

nrule: 6
ndata: 6907
the rank of x's columns:  [5, 0, 4, 1, 2, 3]
COUNT: 100000
COUNT: 200000
COUNT: 300000
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  41.911646127700806
lambda:  0.0035
leaves:  [(4,), (-4, 1), (-4, -3, -1), (-5, -4, -1, 3), (-4, -1, 3, 5)]
prediction:  [1, 1, 0, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  186494
time when the best tree is achieved:  25.88404631614685
TOTAL COUNT:  346501


In [17]:
bbound_similar_when_sub(x6, y, lamb=0.0035, prior_metric="objective", MAXDEPTH = 4)


nrule: 6
ndata: 6907
the rank of x's columns:  [5, 0, 4, 1, 2, 3]
COUNT: 100000
COUNT: 200000
COUNT: 300000
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  43.59049963951111
lambda:  0.0035
leaves:  [(4,), (-4, 1), (-4, -3, -1), (-5, -4, -1, 3), (-4, -1, 3, 5)]
prediction:  [1, 1, 0, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  186494
time when the best tree is achieved:  26.89571261405945
TOTAL COUNT:  346501


In [30]:
bbound_nosimilar_multicopies(x6, y, lamb=0.0035, prior_metric="objective", MAXDEPTH = 4)

nrule: 6
ndata: 6907
the rank of x's columns:  [5, 0, 4, 1, 2, 3]
COUNT: 100000
COUNT: 200000
COUNT: 300000
COUNT: 400000
COUNT: 500000
COUNT: 600000
COUNT: 700000
COUNT: 800000
COUNT: 900000
COUNT: 1000000
COUNT: 1100000
COUNT: 1200000
COUNT: 1300000
COUNT: 1400000
COUNT: 1500000
COUNT: 1600000
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  55.847846269607544
lambda:  0.0035
leaves:  [(4,), (-4, 1), (-4, -3, -1), (-5, -4, -1, 3), (-4, -1, 3, 5)]
prediction:  [1, 1, 0, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  1208310
time when the best tree is achieved:  44.417227268218994
TOTAL COUNT:  1644294


In [31]:
bbound_similar(x6, y, lamb=0.0035, prior_metric="objective", MAXDEPTH = 5)

nrule: 6
ndata: 6907
the rank of x's columns:  [5, 0, 4, 1, 2, 3]
COUNT: 100000
COUNT: 200000
COUNT: 300000
COUNT: 400000
COUNT: 500000
COUNT: 600000
COUNT: 700000
COUNT: 800000
COUNT: 900000
COUNT: 1000000
COUNT: 1100000
COUNT: 1200000
COUNT: 1300000
COUNT: 1400000
COUNT: 1500000
COUNT: 1600000
COUNT: 1700000
COUNT: 1800000
COUNT: 1900000
COUNT: 2000000
COUNT: 2100000
COUNT: 2200000
COUNT: 2300000
COUNT: 2400000
COUNT: 2500000
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  131910.08038282394
lambda:  0.0035
leaves:  [(4,), (-4, 1), (-4, -3, -1), (-5, -4, -1, 3), (-4, -1, 3, 5)]
prediction:  [1, 1, 0, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  1787384
time when the best tree is achieved:  51108.432156562805
TOTAL COUNT:  2593677


In [ ]:
bbound_nosimilar_multicopies(x6, y, lamb=0.0035, prior_metric="objective", MAXDEPTH = 5)

nrule: 6
ndata: 6907
the rank of x's columns:  [5, 0, 4, 1, 2, 3]
COUNT: 100000
COUNT: 200000
COUNT: 300000
COUNT: 400000
COUNT: 500000
COUNT: 600000
COUNT: 700000
COUNT: 800000
COUNT: 900000
COUNT: 1000000
COUNT: 1100000
COUNT: 1200000
COUNT: 1300000
COUNT: 1400000
COUNT: 1500000
COUNT: 1600000
COUNT: 1700000
COUNT: 1800000
COUNT: 1900000
COUNT: 2000000
COUNT: 2100000
COUNT: 2200000
COUNT: 2300000
COUNT: 2400000
COUNT: 2500000
COUNT: 2600000
COUNT: 2700000
COUNT: 2800000
COUNT: 2900000
COUNT: 3000000
COUNT: 3100000
COUNT: 3200000
COUNT: 3300000
COUNT: 3400000
COUNT: 3500000
COUNT: 3600000
COUNT: 3700000
COUNT: 3800000
COUNT: 3900000
COUNT: 4000000
COUNT: 4100000
COUNT: 4200000
COUNT: 4300000
COUNT: 4400000
COUNT: 4500000
COUNT: 4600000
COUNT: 4700000
COUNT: 4800000
COUNT: 4900000
COUNT: 5000000
COUNT: 5100000
COUNT: 5200000
COUNT: 5300000
COUNT: 5400000
COUNT: 5500000
COUNT: 5600000
COUNT: 5700000
COUNT: 5800000
COUNT: 5900000
COUNT: 6000000


In [ ]:
bbound_similar_when_big(x6, y, lamb=0.0035, prior_metric="objective", MAXDEPTH = 5)

In [77]:
bbound_nosimilar(x_mine, y, lamb=0.0035, prior_metric="objective", MAXDEPTH = 4)

nrule: 9
ndata: 6907
the rank of x's columns:  [8, 7, 0, 6, 5, 4, 2, 1, 3]
COUNT: 100000
COUNT: 200000
COUNT: 300000
COUNT: 400000
COUNT: 500000
COUNT: 600000
COUNT: 700000
COUNT: 800000
COUNT: 900000
COUNT: 1000000
COUNT: 1100000
COUNT: 1200000
COUNT: 1300000
COUNT: 1400000
COUNT: 1500000
COUNT: 1600000
COUNT: 1700000
COUNT: 1800000
COUNT: 1900000
COUNT: 2000000
COUNT: 2100000
COUNT: 2200000
COUNT: 2300000
COUNT: 2400000
COUNT: 2500000
COUNT: 2600000
COUNT: 2700000
COUNT: 2800000
COUNT: 2900000
COUNT: 3000000
COUNT: 3100000


KeyboardInterrupt: 

In [51]:
bbound_nosimilar(x_mine, y, lamb=0.0035, prior_metric="objective", MAXDEPTH = 5)

nrule: 6
ndata: 6907
the rank of x's columns:  [2, 4, 5, 1, 0, 3]
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  20.07693123817444
lambda:  0.0035
leaves:  [(5,), (-5, -2), (-5, 2, 6), (-6, -5, -3, 2), (-6, -5, 2, 3)]
prediction:  [0, 1, 0, 1, 0]
Objective:  0.44011473867091355
COUNT of the best tree:  36112
time when the best tree is achieved:  13.207933902740479
TOTAL COUNT:  96171


In [17]:
bbound_similar(x, y, lamb=0.0035, prior_metric="objective", MAXDEPTH = 4)

nrule: 5
ndata: 6907
the rank of x's columns:  [4, 0, 1, 2, 3]
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  7.832329988479614
lambda:  0.0035
leaves:  [(3,), (-3, 1), (-5, -3, -1), (-4, -3, -1, 5), (-3, -1, 4, 5)]
prediction:  [1, 1, 0, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  1794
time when the best tree is achieved:  3.0771291255950928
TOTAL COUNT:  6161


In [16]:
### priority tree.risk
bbound_similar_priority(x, y, lamb=0.0035, prior_metric="objective", MAXDEPTH = 4)

nrule: 5
ndata: 6907
the rank of x's columns:  [4, 0, 1, 2, 3]
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  9.078894138336182
lambda:  0.0035
leaves:  [(3,), (-3, 1), (-5, -3, -1), (-4, -3, -1, 5), (-3, -1, 4, 5)]
prediction:  [1, 1, 0, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  1794
time when the best tree is achieved:  3.065239667892456
TOTAL COUNT:  6161


In [16]:
### priority -(simil_ct+1)
bbound_similar_priority(x, y, lamb=0.0035, prior_metric="objective", MAXDEPTH = 4)

nrule: 5
ndata: 6907
the rank of x's columns:  [4, 0, 1, 2, 3]
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  8.36022663116455
lambda:  0.0035
leaves:  [(3,), (-3, 1), (-5, -3, -1), (-4, -3, -1, 5), (-3, -1, 4, 5)]
prediction:  [1, 1, 0, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  1794
time when the best tree is achieved:  3.0293593406677246
TOTAL COUNT:  6161


In [19]:
aaa = []

In [20]:
heapq.heappush(aaa, (0.1, (1, 2)))

In [21]:
aaa

[(0.1, (1, 2))]

In [22]:
(0.1, (1, 2)) in aaa

True

In [21]:
cProfile.run("bbound_nosimilar(x, y, lamb=0.0035, prior_metric=\"objective\", MAXDEPTH = 4)")

nrule: 5
ndata: 6907
the rank of x's columns:  [4, 0, 1, 2, 3]
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  10.082534074783325
lambda:  0.0035
leaves:  [(3,), (-3, 1), (-5, -3, -1), (-4, -3, -1, 5), (-3, -1, 4, 5)]
prediction:  [1, 1, 0, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  2927
time when the best tree is achieved:  3.949885606765747
TOTAL COUNT:  12764
         1033248 function calls (1033246 primitive calls) in 10.233 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        5    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:997(_handle_fromlist)
        1    0.001    0.001   10.233   10.233 <string>:1(<module>)
        3    0.000    0.000    0.000    0.000 __init__.py:200(iteritems)
        1    0.000    0.000    0.000    0.000 _bootlocale.py:23(getpreferredencoding)
    29392    0.107   

In [22]:
cProfile.run("bbound_similar(x, y, lamb=0.0035, prior_metric=\"objective\", MAXDEPTH = 4)")

nrule: 5
ndata: 6907
the rank of x's columns:  [4, 0, 1, 2, 3]
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  53.315651416778564
lambda:  0.0035
leaves:  [(3,), (-3, 1), (-5, -3, -1), (-4, -3, -1, 5), (-3, -1, 4, 5)]
prediction:  [1, 1, 0, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  2916
time when the best tree is achieved:  5.6540985107421875
TOTAL COUNT:  9157
         9280221 function calls (9280219 primitive calls) in 53.476 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        5    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:997(_handle_fromlist)
        1    0.001    0.001   53.476   53.476 <string>:1(<module>)
        3    0.000    0.000    0.000    0.000 __init__.py:200(iteritems)
        1    0.000    0.000    0.000    0.000 _bootlocale.py:23(getpreferredencoding)
    19345    0.071   

        3    0.000    0.000    0.000    0.000 {method 'join' of 'str' objects}
     9158    0.023    0.000    0.023    0.000 {method 'pop' of 'list' objects}
    19346    0.172    0.000    0.172    0.000 {method 'reduce' of 'numpy.ufunc' objects}
      194    0.001    0.000    0.001    0.000 {method 'reshape' of 'numpy.ndarray' objects}
        1    0.000    0.000    0.000    0.000 {method 'search' of '_sre.SRE_Pattern' objects}
      578    1.506    0.003    1.506    0.003 {method 'sub' of '_sre.SRE_Pattern' objects}
    19345    0.093    0.000    0.337    0.000 {method 'sum' of 'numpy.ndarray' objects}
        1    0.000    0.000    0.000    0.000 {method 'tolist' of 'numpy.ndarray' objects}
        1    0.000    0.000    0.000    0.000 {method 'update' of 'dict' objects}
        2    0.000    0.000    0.000    0.000 {method 'write' of '_io.TextIOWrapper' objects}
        1    0.000    0.000    0.000    0.000 {pandas._libs.algos.rank_1d_float64}
        2    0.001    0.001    0.001  

In [23]:
bbound_nosimilar(x, y, lamb=0.0035, prior_metric="objective", MAXDEPTH = 5)

nrule: 5
ndata: 6907
the rank of x's columns:  [4, 0, 1, 2, 3]
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  11.018238544464111
lambda:  0.0035
leaves:  [(3,), (-3, 1), (-5, -3, -1), (-4, -3, -1, 5), (-3, -1, 4, 5)]
prediction:  [1, 1, 0, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  14826
time when the best tree is achieved:  4.4402501583099365
TOTAL COUNT:  58505


In [18]:
### tighter incremental ###
bbound_nosimilar(x, y, lamb=0.0035, prior_metric="objective", MAXDEPTH = 5)

nrule: 5
ndata: 6907
the rank of x's columns:  [4, 0, 1, 2, 3]
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  6.423090696334839
lambda:  0.0035
leaves:  [(3,), (-3, 1), (-5, -3, -1), (-4, -3, -1, 5), (-3, -1, 4, 5)]
prediction:  [1, 1, 0, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  5525
time when the best tree is achieved:  3.6138153076171875
TOTAL COUNT:  18855


In [24]:
bbound_similar(x, y, lamb=0.0035, prior_metric="objective", MAXDEPTH = 5)

nrule: 5
ndata: 6907
the rank of x's columns:  [4, 0, 1, 2, 3]
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  319.85523533821106
lambda:  0.0035
leaves:  [(3,), (-3, 1), (-5, -3, -1), (-4, -3, -1, 5), (-3, -1, 4, 5)]
prediction:  [1, 1, 0, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  14056
time when the best tree is achieved:  16.275825262069702
TOTAL COUNT:  48117


In [19]:
### tighter incremental ###
bbound_similar(x, y, lamb=0.0035, prior_metric="objective", MAXDEPTH = 5)

nrule: 5
ndata: 6907
the rank of x's columns:  [4, 0, 1, 2, 3]
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  91.42455744743347
lambda:  0.0035
leaves:  [(3,), (-3, 1), (-5, -3, -1), (-4, -3, -1, 5), (-3, -1, 4, 5)]
prediction:  [1, 1, 0, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  5525
time when the best tree is achieved:  6.0132012367248535
TOTAL COUNT:  20197


In [19]:
bbound_similar(x, y, lamb=0.0035, prior_metric="objective", MAXDEPTH = 5)

nrule: 5
ndata: 6907
the rank of x's columns:  [4, 0, 1, 2, 3]
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  54.678985834121704
lambda:  0.0035
leaves:  [(3,), (-3, 1), (-5, -3, -1), (-4, -3, -1, 5), (-3, -1, 4, 5)]
prediction:  [1, 1, 0, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  5525
time when the best tree is achieved:  5.865641832351685
TOTAL COUNT:  17633


In [18]:
### priority tree.risk
bbound_similar_priority(x, y, lamb=0.0035, prior_metric="objective", MAXDEPTH = 5)

nrule: 5
ndata: 6907
the rank of x's columns:  [4, 0, 1, 2, 3]
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  80.14964938163757
lambda:  0.0035
leaves:  [(3,), (-3, 1), (-5, -3, -1), (-4, -3, -1, 5), (-3, -1, 4, 5)]
prediction:  [1, 1, 0, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  5525
time when the best tree is achieved:  7.568428039550781
TOTAL COUNT:  17633


In [17]:
### priority -(simil_ct+1)
bbound_similar_priority(x, y, lamb=0.0035, prior_metric="objective", MAXDEPTH = 5)

nrule: 5
ndata: 6907
the rank of x's columns:  [4, 0, 1, 2, 3]
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  63.66035604476929
lambda:  0.0035
leaves:  [(3,), (-3, 1), (-5, -3, -1), (-4, -3, -1, 5), (-3, -1, 4, 5)]
prediction:  [1, 1, 0, 0, 1]
Objective:  0.3541150282322281
COUNT of the best tree:  5525
time when the best tree is achieved:  6.575494050979614
TOTAL COUNT:  17633


In [19]:
bbound_nosimilar(x6, y, lamb=0.01, prior_metric="objective", MAXDEPTH = 5)

nrule: 6
ndata: 6907
the rank of x's columns:  [5, 0, 4, 1, 2, 3]
COUNT: 100000
COUNT: 200000
COUNT: 300000
COUNT: 400000
COUNT: 500000
COUNT: 600000
COUNT: 700000
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  104.39202117919922
lambda:  0.01
leaves:  [(1,), (-6, -1), (-1, 6)]
prediction:  [1, 1, 0]
Objective:  0.3748675256985667
COUNT of the best tree:  11
time when the best tree is achieved:  0.5258162021636963
TOTAL COUNT:  754374


In [49]:
### tighter incremental ###
bbound_nosimilar(x6, y, lamb=0.01, prior_metric="objective", MAXDEPTH = 5)

nrule: 6
ndata: 6907
the rank of x's columns:  [5, 0, 4, 1, 2, 3]
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  10.628950119018555
lambda:  0.01
leaves:  [(1,), (-6, -1), (-1, 6)]
prediction:  [1, 1, 0]
Objective:  0.3748675256985667
COUNT of the best tree:  11
time when the best tree is achieved:  0.49529409408569336
TOTAL COUNT:  18104


In [17]:
### tighter incremental ###
bbound_nosimilar(x6, y, lamb=0.01, prior_metric="objective", MAXDEPTH = 6)

nrule: 6
ndata: 6907
the rank of x's columns:  [5, 0, 4, 1, 2, 3]
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  11.318727493286133
lambda:  0.01
leaves:  [(1,), (-6, -1), (-1, 6)]
prediction:  [1, 1, 0]
Objective:  0.3748675256985667
COUNT of the best tree:  11
time when the best tree is achieved:  0.4984321594238281
TOTAL COUNT:  20382


In [18]:
### tighter incremental ###
bbound_similar(x6, y, lamb=0.01, prior_metric="objective", MAXDEPTH = 6)

nrule: 6
ndata: 6907
the rank of x's columns:  [5, 0, 4, 1, 2, 3]
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  54.17368698120117
lambda:  0.01
leaves:  [(1,), (-6, -1), (-1, 6)]
prediction:  [1, 1, 0]
Objective:  0.3748675256985667
COUNT of the best tree:  11
time when the best tree is achieved:  0.4591836929321289
TOTAL COUNT:  11621


In [20]:
### tighter incremental ###
bbound_similar(x6, y, lamb=0.01, prior_metric="objective", MAXDEPTH = 6)

nrule: 6
ndata: 6907
the rank of x's columns:  [5, 0, 4, 1, 2, 3]
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  49.53106093406677
lambda:  0.01
leaves:  [(1,), (-6, -1), (-1, 6)]
prediction:  [1, 1, 0]
Objective:  0.3748675256985667
COUNT of the best tree:  11
time when the best tree is achieved:  0.5225584506988525
TOTAL COUNT:  10780


In [17]:
### priority tree.risk
bbound_similar_priority(x6, y, lamb=0.01, prior_metric="objective", MAXDEPTH = 6)

nrule: 6
ndata: 6907
the rank of x's columns:  [5, 0, 4, 1, 2, 3]
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  67.3289864063263
lambda:  0.01
leaves:  [(1,), (-6, -1), (-1, 6)]
prediction:  [1, 1, 0]
Objective:  0.3748675256985667
COUNT of the best tree:  11
time when the best tree is achieved:  0.5085046291351318
TOTAL COUNT:  10780


In [18]:
### priority -(simil_ct+1)
bbound_similar_priority(x6, y, lamb=0.01, prior_metric="objective", MAXDEPTH = 6)

nrule: 6
ndata: 6907
the rank of x's columns:  [5, 0, 4, 1, 2, 3]
>>> log: False
>>> support bound: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
total time:  60.29576778411865
lambda:  0.01
leaves:  [(1,), (-6, -1), (-1, 6)]
prediction:  [1, 1, 0]
Objective:  0.3748675256985667
COUNT of the best tree:  11
time when the best tree is achieved:  0.49385738372802734
TOTAL COUNT:  10780


In [ ]:
### tighter incremental ###
bbound_nosimilar(x6, y, lamb=0.0036, prior_metric="objective", MAXDEPTH = 6)

nrule: 6
ndata: 6907
the rank of x's columns:  [5, 0, 4, 1, 2, 3]
COUNT: 100000
COUNT: 200000
COUNT: 300000
COUNT: 400000
COUNT: 500000
COUNT: 600000
COUNT: 700000
COUNT: 800000
COUNT: 900000
COUNT: 1000000
COUNT: 1100000
COUNT: 1200000
COUNT: 1300000
COUNT: 1400000
COUNT: 1500000
COUNT: 1600000
COUNT: 1700000
COUNT: 1800000
COUNT: 1900000
COUNT: 2000000
COUNT: 2100000
COUNT: 2200000
COUNT: 2300000
COUNT: 2400000
COUNT: 2500000
COUNT: 2600000
COUNT: 2700000
COUNT: 2800000
COUNT: 2900000
COUNT: 3000000
COUNT: 3100000
COUNT: 3200000
COUNT: 3300000
COUNT: 3400000
COUNT: 3500000
COUNT: 3600000
COUNT: 3700000
COUNT: 3800000
COUNT: 3900000
COUNT: 4000000
COUNT: 4100000
COUNT: 4200000
COUNT: 4300000
COUNT: 4400000
COUNT: 4500000
COUNT: 4600000
COUNT: 4700000
COUNT: 4800000
COUNT: 4900000
COUNT: 5000000
COUNT: 5100000
COUNT: 5200000
COUNT: 5300000
COUNT: 5400000
COUNT: 5500000
COUNT: 5600000
COUNT: 5700000
COUNT: 5800000
COUNT: 5900000
COUNT: 6000000
COUNT: 6100000
COUNT: 6200000
COUNT: 630000

In [ ]:
### tighter incremental ###
bbound_similar(x6, y, lamb=0.0035, prior_metric="objective", MAXDEPTH = 6)